# Excercises 
# 1. Tune the network
Run the experiment below, explore the different parameters (see suggestions below) and study the result with tensorboard. 
Make a single page (1 a4) report of your findings. Use your visualisation skills to communicate your most important findings.

In [ ]:
from mads_datasets import DatasetFactoryProvider, DatasetType

from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics

import torch.optim as optim
from torch import nn
from tomlserializer import TOMLSerializer

In [ ]:
import torch

print("torch version:", torch.__version__)
print("torch cuda version:", torch.version.cuda)
print("cuda available:", torch.cuda.is_available())
print("cuda device count:", torch.cuda.device_count())

In [ ]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

We will be using `tomlserializer` to easily keep track of our experiments, and to easily save the different things we did during our experiments.
It can export things like settings and models to a simple `toml` file, which can be easily shared, checked and modified.

First, we need the data. 

In [ ]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

We need a way to determine how well our model is performing. We will use accuracy as a metric.

In [ ]:
accuracy = metrics.Accuracy()

You can set up a single experiment.

- We will show the model batches of 64 images, 
- and for every epoch we will show the model 100 batches (trainsteps=100).
- then, we will test how well the model is doing on unseen data (teststeps=100).
- we will report our results during training to tensorboard, and report all configuration to a toml file.
- we will log the results into a directory called "modellogs", but you could change this to whatever you want.

In [ ]:
import torch
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)


We will use a very basic model: a model with three linear layers.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(
    num_classes=10, units1=256, units2=256)

I developped the `tomlserializer` package, it is a useful tool to save configs, models and settings as a tomlfile; that way it is easy to track what you changed during your experiments.

This package will 1. check if there is a `__dict__` attribute available, and if so, it will use that to extract the parameters that do not start with an underscore, like this:

In [ ]:
{k: v for k, v in model.__dict__.items() if not k.startswith("_")}

This means that if you want to add more parameters to the `.toml` file, eg `units3`, you can add them to the class like this:

```python
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int, units3: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.units3 = units3  # <-- add this line
```

And then it will be added to the `.toml` file. Check the result for yourself by using the `.save()` method of the `TomlSerializer` class like this:

In [ ]:
tomlserializer = TOMLSerializer()
tomlserializer.save(settings, "settings.toml")
tomlserializer.save(model, "model.toml")

Check the `settings.toml` and `model.toml` files to see what is in there.

## Script for looping through some epochs

In [ ]:
import torch

units = [64, 32, 16]
loss_fn = torch.nn.CrossEntropyLoss()

main_folder = "modellogs"
subfolder = "change_epochs"
amount_of_epochs = [5, 8, 10]

for epochs in amount_of_epochs:
    epoch_subfolder = f"{subfolder}/epochs_{epochs}"
    settings = TrainerSettings(
        epochs=epochs,
        metrics=[accuracy],
        logdir=f"{main_folder}/{epoch_subfolder}",
        train_steps=len(train),
        valid_steps=len(valid),
        reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
    )

    for unit1 in units:
        for unit2 in units:
            if unit2 <= unit1:
                print(f"Epochs: {epochs}, Units: {unit1}, {unit2}")
                model = NeuralNetwork(num_classes=10, units1=unit1, units2=unit2)

                trainer = Trainer(
                    model=model,
                    settings=settings,
                    loss_fn=loss_fn,
                    optimizer=optim.Adam,
                    traindataloader=trainstreamer,
                    validdataloader=validstreamer,
                    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
                )
                trainer.loop()

## Script for changing units

In [ ]:
import torch

units = [2**i for i in range(4, 11)]  # 16, 32, 64, ..., 1024
loss_fn = torch.nn.CrossEntropyLoss()

main_folder = "modellogs"
subfolder = "change_units"

for unit1 in units:
    for unit2 in units:
        if unit2 <= unit1:
            run_subfolder = f"{subfolder}/units_{unit1}_{unit2}"
            settings = TrainerSettings(
                epochs=3,
                metrics=[accuracy],
                logdir=f"{main_folder}/{run_subfolder}",
                train_steps=len(train),
                valid_steps=len(valid),
                reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
            )
            print(f"Units: {unit1}, {unit2}")
            model = NeuralNetwork(num_classes=10, units1=unit1, units2=unit2)

            trainer = Trainer(
                model=model,
                settings=settings,
                loss_fn=loss_fn,
                optimizer=optim.Adam,
                traindataloader=trainstreamer,
                validdataloader=validstreamer,
                scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            )
            trainer.loop()

## Script for changing batchsize

In [ ]:
import torch

batchsizes = [4,32,128]
units = [64, 32, 16]
loss_fn = torch.nn.CrossEntropyLoss()
main_folder = "modellogs"
subfolder = "change_batchsize"

for batchsize in batchsizes:
    print(f"Running with batchsize: {batchsize}")
    fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
    preprocessor = BasePreprocessor()
    streamers = fashionfactory.create_datastreamer(
        batchsize=batchsize, preprocessor=preprocessor
    )
    train = streamers["train"]
    valid = streamers["valid"]
    trainstreamer = train.stream()
    validstreamer = valid.stream()

    for unit1 in units:
        for unit2 in units:
            if unit2 <= unit1:
                run_subfolder = (
                    f"{subfolder}/batchsize_{batchsize}_units_{unit1}_{unit2}"
                )
                settings = TrainerSettings(
                    epochs=3,
                    metrics=[accuracy],
                    logdir=f"{main_folder}/{run_subfolder}",
                    train_steps=len(train),
                    valid_steps=len(valid),
                    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
                )
                print(f"Batchsize: {batchsize}, Units: {unit1}, {unit2}")
                model = NeuralNetwork(num_classes=10, units1=unit1, units2=unit2)

                trainer = Trainer(
                    model=model,
                    settings=settings,
                    loss_fn=loss_fn,
                    optimizer=optim.Adam,
                    traindataloader=trainstreamer,
                    validdataloader=validstreamer,
                    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
                )
                trainer.loop()

## Script for changing depth model

In [ ]:
import torch

fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int, units3: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.units3 = units3
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, units3),
            nn.ReLU(),
            nn.Linear(units3, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


units = [128, 64, 32, 16]
loss_fn = torch.nn.CrossEntropyLoss()

main_folder = "modellogs"
subfolder = "change_depth"

for unit1 in units:
    for unit2 in units:
        if unit2 <= unit1:
            for unit3 in units:
                if unit3 <= unit2:
                    run_subfolder = f"{subfolder}/units_{unit1}_{unit2}_{unit3}"
                    settings = TrainerSettings(
                        epochs=3,
                        metrics=[accuracy],
                        logdir=f"{main_folder}/{run_subfolder}",
                        train_steps=len(train),
                        valid_steps=len(valid),
                        reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
                    )
                    print(f"Units: {unit1}, {unit2}, {unit3}")
                    model = NeuralNetwork(
                        num_classes=10, units1=unit1, units2=unit2, units3=unit3
                    )

                    trainer = Trainer(
                        model=model,
                        settings=settings,
                        loss_fn=loss_fn,
                        optimizer=optim.Adam,
                        traindataloader=trainstreamer,
                        validdataloader=validstreamer,
                        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
                    )
                    trainer.loop()

## Script for changing learning rate

In [ ]:
import torch


class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


units = [64, 32, 16]
loss_fn = torch.nn.CrossEntropyLoss()
main_folder = "modellogs"
subfolder = "change_learningrate"
learningrates = [1e-2, 1e-3, 1e-4, 1e-5]

for lr in learningrates:
    for unit1 in units:
        for unit2 in units:
            if unit2 <= unit1:
                run_subfolder = f"{subfolder}/lr_{lr}_units_{unit1}_{unit2}"
                settings = TrainerSettings(
                    epochs=3,
                    metrics=[accuracy],
                    logdir=f"{main_folder}/{run_subfolder}",
                    train_steps=len(train),
                    valid_steps=len(valid),
                    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
                    optimizer_kwargs={"lr": lr},
                )
                print(f"Learning rate: {lr}, Units: {unit1}, {unit2}")
                model = NeuralNetwork(num_classes=10, units1=unit1, units2=unit2)

                trainer = Trainer(
                    model=model,
                    settings=settings,
                    loss_fn=loss_fn,
                    optimizer=optim.Adam,
                    traindataloader=trainstreamer,
                    validdataloader=validstreamer,
                    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
                )
                trainer.loop()

## Script for changing optimizer

In [ ]:
import torch

units = [64, 32, 16]
loss_fn = torch.nn.CrossEntropyLoss()
main_folder = "modellogs"
subfolder = "change_optimizer"

learning_rate = 0.01

optimizers = {
    "SGD": optim.SGD,
    "Adam": optim.Adam,
    "RMSprop": optim.RMSprop,
}

learning_rate = 0.01

for opt_name, opt_class in optimizers.items():
    for unit1 in units:
        for unit2 in units:
            if unit2 <= unit1:
                run_subfolder = f"{subfolder}/{opt_name}_units_{unit1}_{unit2}"
                settings = TrainerSettings(
                    epochs=3,
                    metrics=[accuracy],
                    logdir=f"{main_folder}/{run_subfolder}",
                    train_steps=len(train),
                    valid_steps=len(valid),
                    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
                    optimizer_kwargs={"lr": learning_rate},
                )
                print(f"Optimizer: {opt_name}, Units: {unit1}, {unit2}")
                model = NeuralNetwork(num_classes=10, units1=unit1, units2=unit2)

                trainer = Trainer(
                    model=model,
                    settings=settings,
                    loss_fn=loss_fn,
                    optimizer=opt_class,  
                    traindataloader=trainstreamer,
                    validdataloader=validstreamer,
                    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
                )
                trainer.loop()

Because we have set the ReportType to TOML, you will find in every log dir a model.toml and settings.toml file.

Run the experiment, and study the result with tensorboard. 

Locally, it is easy to do that with VS code itself. On the server, you have to take these steps:

- in the terminal, `cd` to the location of the repository
- activate the python environment for the shell. This can be done with `.venv\Scripts\activate`.
- run `tensorboard --logdir=1-hypertuning-gridsearch/modellogs` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`